# MAV Dynamics

- Chapter 3.

In [169]:
import numpy as np
from math import cos, sin, tan, pi
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


In [153]:
# Inertia Matrix J
MOMENT_OF_INERTIA = [10, 10, 10]   # Jx, Jy, Jz
PRODUCTS_OF_INERTIA = [0, 1, 0] # Jxy=0, Jxz, Jyz=0
def get_inertia_matrix(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA):
    Jx, Jy, Jz = np.reshape(MOMENT_OF_INERTIA, (3))
    Jxy, Jxz, Jyz = np.reshape(PRODUCTS_OF_INERTIA, (3))
    return [[Jx, -Jxy, -Jxz],
            [-Jxy, Jy, -Jyz],
            [-Jxz, -Jyz, Jz]]

def get_Gamma_components(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA):
    Jx, Jy, Jz = np.reshape(MOMENT_OF_INERTIA, (3))
    Jxy, Jxz, Jyz = np.reshape(PRODUCTS_OF_INERTIA, (3))
    Gamma = Jx*Jz - Jxz*Jxz
    Gamma1 = Jxz*(Jx - Jy + Jz) / Gamma
    Gamma2 = (Jz*(Jz-Jy) + Jxz*Jxz) / Gamma
    Gamma3 = Jz / Gamma
    Gamma4 = Jxz / Gamma
    Gamma5 = (Jz - Jx) / Jy
    Gamma6 = Jxz / Jy
    Gamma7 = ((Jx-Jy)*Jx + Jxz*Jxz) / Gamma
    Gamma8 = Jx / Gamma
    return Gamma, (Gamma1, Gamma2, Gamma3, Gamma4, Gamma5, Gamma6, Gamma7, Gamma8)
    
J = get_inertia_matrix(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA)
Gamma, Gamma_i = get_Gamma_components(MOMENT_OF_INERTIA, PRODUCTS_OF_INERTIA)

In [154]:
# States

def get_rot_b2v(euler): # rot matrix from body frame to inertial frame
    phi, theta, psi = np.reshape(euler,(3))
    row1 = [cos(theta)*cos(psi), sin(phi)*sin(theta)*cos(psi) - cos(phi)*sin(psi), cos(phi)*sin(theta)*cos(psi) + sin(phi)*sin(psi)]
    row2 = [cos(theta)*sin(psi), sin(phi)*sin(theta)*sin(psi) + cos(phi)*cos(psi), cos(phi)*sin(theta)*sin(psi) - sin(phi)*cos(psi)]
    row3 = [-sin(theta), sin(phi)*cos(theta), cos(phi)*cos(theta)]
    return np.array([row1, row2, row3])
    
def get_ned_dot(uvw, euler, **kwargs): # kwargs: wind_ned
    # Eq.(3.14) or (3.18)
    rot_b2v = get_rot_b2v(euler)
    ned_dot = np.matmul(rot_b2v, np.reshape(uvw,(3,1))) 
    if 'wind_ned' in kwargs.keys():
        wind_ned = kwargs['wind_ned']
        ned_dot = ned_dot + np.reshape(wind_ned,(3,1))
    return ned_dot

def get_uvw_dot(uvw, pqr, forces, MASS, **kwargs): # kwargs: wind_ned_dot, euler
    # Eq. (3.15) or (3.19)
    p, q, r = np.reshape(pqr, (3))
    u, v, w = np.reshape(uvw, (3))
    term1 = np.reshape([r*v - q*w, p*w - r*u, q*u - p*v],(3,1))
    term2 = 1/MASS*np.reshape(forces,(3,1))
    if ('wind_ned_dot' in kwargs.keys()) or ('euler' in kwargs.keys()):
        try:
            wind_ned_dot = kwargs['wind_ned_dot']
            euler = kwargs['euler']
            rot_v2b = np.transpose(get_rot_b2v(euler))
            term2 = term2 + np.matmul(rot_v2b, np.reshape(wind_ned_dot,(3,1)))
        except KeyError as ne:
            print(f'Key {ne} not found. Passed arguments {list(kwargs.keys())} is ignored. Compute uvw in ground speed..')
        except:
            pass
    return term1 + term2

def get_euler_dot(euler, pqr):
    # Eq. (3.16)
    phi, theta, _ = np.reshape(euler,(3))
    row1 = [1, sin(phi)*tan(theta), cos(phi)*tan(theta)]
    row2 = [0, cos(phi), -sin(phi)]
    row3 = [0, sin(phi)/cos(theta), cos(phi)/cos(theta)]
    return np.matmul([row1, row2, row3], np.reshape(pqr,(3,1)))

def get_pqr_dot(pqr, moments, Jy, Gamma_i):
    # Eq. (3.17)
    p, q, r = np.reshape(pqr, (3))
    l, m, n = np.reshape(moments, (3))
    G1, G2, G3, G4, G5, G6, G7, G8 = Gamma_i
    term1 = np.array([[G1*p*q - G2*q*r], [G5*p*r - G6*(p**2-r**2)], [G7*p*q - G1*q*r]])
    term2 = np.array([[G3*l + G4*n], [m/Jy], [G4*l + G8*n]])
    return term1 + term2

def get_dynamics(states, MASS, MOMENT_OF_INERTIA, Gamma_i, forces, moments, **kwargs):
    _, uvw, euler, pqr = states
    ned_dot = get_ned_dot(uvw, euler, **kwargs)
    if 'wind_ned_dot' in kwargs.keys():
        kwargs['euler'] = euler
    uvw_dot = get_uvw_dot(uvw, pqr, forces, MASS, **kwargs)
    euler_dot = get_euler_dot(euler, pqr)
    pqr_dot = get_pqr_dot(pqr, moments, MOMENT_OF_INERTIA[1], Gamma_i)
    return ned_dot, uvw_dot, euler_dot, pqr_dot

Chapter 4.

---

**Stability Derivatives**

The areodynamic coefficients $C_{m_\alpha}, C_{l_\beta}, C_{n_\beta}, C_{m_q}, C_{l_p}$, and $C_{n_r}$ are referred to as *stability derivatives* because their values determine the static and dynamic stability of the MAV.

- $C_{m_\alpha}$ is referred to as the longitudinal static stability derivative. For the MAV to be statically stable, $C_{m_\alpha}$ must be less than zero.
- $C_{l_\beta}$ is called the roll static stability derivative and is typically associated with dihedral in the wings. For static stability in roll, $C_{l_\beta}$  must be negative.
- $C_{n_\beta}$ is referred to as the yaw static stability derivative and is sometimes called the weathercock stability derivative. For the MAV to be stable in yaw, $C_{n_beta}$ must be positive.

---
---

**Damping Derivatives**

Each of the damping derivatives is usually negative.

- $C_{m_q}$ is referred to as the pitch damping derivative.
- $C_{l_p}$ is called the roll damping derivative.
- $C_{n_r}$ is referred to as the yaw damping derivative.

---
---

**Control Derivatives**

The areodynamic coefficients $C_{m_{\delta_e}}, C_{l_{\delta_a}}$, and $C_{n_{\delta_r}}$ are associated with the deflection of control surfaces and are referred to as the *primary control derivatives*. They are primary because the moments produced are the intended result of the specific control surface deflection.

$C_{l_{\delta_r}}$ and $C_{n_{\delta_a}}$ are called *cross-control derivatives*. They define the off-axis moments that occur when the control surfaces are deflected.

---

In [192]:
# Longitudinal Aerodynamic Coefficients
LIFT_COEFFICIENTS = [1, 1, 0, 1] # [C_L_0, C_L_alpha, C_L_q, C_L_delta_e]
DRAG_COEFFICIENTS = [1, 1, 0, 1] # [C_D_0, C_D_alpha, C_D_q, C_D_delta_e]
PITCHING_MOMENT_COEFFICIENTS = [1, -1, -1, 1] # [C_m_0, C_m_alpha, C_m_q, C_m_delta_e]
# Lateral Aerodynamic Coefficients
LATERAL_FORCE_COEFFICIENTS = [1, 1, 1, 1, 1, 1] # [C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a, C_Y_delta_r]
ROLL_MOMENT_COEFFICIENTS = [1, -1, -1, 1, 1, 1] # [C_l_0, C_l_beta, C_l_p, C_l_R, C_l_delta_a, C_l_delta_r]
YAW_MOMENT_COEFFICIENTS = [1, 1, 1, -1, 1, 1] # [C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a, C_n_delta_r]

lon_linear_coef = [LIFT_COEFFICIENTS, DRAG_COEFFICIENTS, PITCHING_MOMENT_COEFFICIENTS]
lat_linear_coef = [LATERAL_FORCE_COEFFICIENTS, ROLL_MOMENT_COEFFICIENTS, YAW_MOMENT_COEFFICIENTS]

def get_aerodynamic_coefficients(lon_linear_coef, lat_linear_coef, MEAN_CHORD, WINGSPAN, AOA, sideslip, airspeed, pqr, control_surface):
    CL_coef, CD_coef, Cm_coef = lon_linear_coef
    CY_coef, Cl_coef, Cn_coef = lat_linear_coef
    p, q, r = np.reshape(pqr, (3))
    delta_e, delta_a, delta_r = np.reshape(control_surface, (3))

    b = WINGSPAN
    c = MEAN_CHORD
    alpha = AOA
    beta = sideslip
    Va = airspeed

    CL = np.matmul(np.reshape(CL_coef,(1,4)), np.reshape([1, alpha, 0.5*c*q/Va, delta_e],(4,1)))
    CD = np.matmul(np.reshape(CD_coef,(1,4)), np.reshape([1, alpha, 0.5*c*q/Va, delta_e],(4,1)))
    Cm = np.matmul(np.reshape(Cm_coef,(1,4)), np.reshape([1, alpha, 0.5*c*q/Va, delta_e],(4,1)))
    CY = np.matmul(np.reshape(CY_coef,(1,6)), np.reshape([1, beta, 0.5*b*p/Va, 0.5*b*r/Va, delta_a, delta_r],(6,1)))
    Cl = np.matmul(np.reshape(Cl_coef,(1,6)), np.reshape([1, beta, 0.5*b*p/Va, 0.5*b*r/Va, delta_a, delta_r],(6,1)))
    Cn = np.matmul(np.reshape(Cn_coef,(1,6)), np.reshape([1, beta, 0.5*b*p/Va, 0.5*b*r/Va, delta_a, delta_r],(6,1)))

    lon_coef = list(np.reshape([CL, CD, Cm], (3)))
    lat_coef = list(np.reshape([CY, Cl, Cn], (3)))
    return [lon_coef, lat_coef]

In [193]:
# Aerodynamic Forces and Moments
def get_longitudinal_body_forces(F_lift, F_drag, AOA):
    mtx = [[cos(AOA), -sin(AOA)], [sin(AOA), cos(AOA)]]
    return np.matmul(mtx, np.reshape([-F_drag,-F_lift], (2))) # (fx, fz) in body frame

def longitudinal_aerodynamics(AIR_DENSITY, PLATFORM_AREA, MEAN_CHORD, AOA, airspeed, lon_coef):
    # Eq. (4.2)-(4.5)
    CL, CD, Cm = np.reshape(lon_coef, (3))
    F_lift = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*CL
    F_drag = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*CD
    m = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*MEAN_CHORD*Cm
    fx, fz = get_longitudinal_body_forces(F_lift, F_drag, AOA)
    return fx, fz, m

def lateral_aerodynamics(AIR_DENSITY, PLATFORM_AREA, WINGSPAN, airspeed, lat_coef):
    # Page 50
    CY, Cl, Cn = np.reshape(lat_coef, (3))
    fy = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*CY
    l = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*WINGSPAN*Cl
    n = 0.5*AIR_DENSITY*airspeed**2*PLATFORM_AREA*WINGSPAN*Cn
    return fy, l, n


# Gravity force
def get_gravity_force(GRAV_ACC, MASS, euler):
    phi, theta, psi = np.reshape(euler,(3))
    fx = -MASS*GRAV_ACC*sin(theta)
    fy = MASS*GRAV_ACC*cos(theta)*sin(phi)
    fz = MASS*GRAV_ACC*cos(theta)*cos(phi)
    return fx, fy, fz


# Propeller force and moment
def get_propeller_force(AIR_DENSITY, SWEPT, PROP_CHORD, MOTOR_FORCE_COEF, airspeed, delta_t):
    fx = 0.5*AIR_DENSITY*SWEPT*PROP_CHORD*((MOTOR_FORCE_COEF*delta_t)**2 - airspeed**2)
    fy, fz = 0, 0
    return fx, fy, fz

def get_propeller_moment(MOTOR_TORQUE_COEF, PROP_SPEED_COEF, delta_t):
    l = -MOTOR_TORQUE_COEF*(PROP_SPEED_COEF*delta_t)**2
    return l, m, n

In [204]:
GRAV_ACC = 9.8
PLATFORM_AREA = 100 # S
MEAN_CHORD = 4 # c
WINGSPAN = 30 # b
AIR_DENSITY = 1 # rho
PROP_CHORD = 2
SWEPT = pi*PROP_CHORD**2
MOTOR_FORCE_COEF = 2
MOTOR_TORQUE_COEF = 2
PROP_SPEED_COEF = 1
MASS = 100

params = dotdict()
params.GRAV_ACC = 9.8
params.PLATFORM_AREA = 100 # S
params.MEAN_CHORD = 4 # c
params.WINGSPAN = 30
params.AIR_DENSITY = 1 # rho
params.PROP_CHORD = 2
params.SWEPT = pi*PROP_CHORD**2
params.MOTOR_FORCE_COEF = 2
params.MOTOR_TORQUE_COEF = 2
params.PROP_SPEED_COEF = 1
params.Gamma_i = Gamma_i
params.MOMENT_OF_INERTIA = MOMENT_OF_INERTIA
params.MASS = MASS

alpha = pi/10
beta = 0
airspeed = 100
control_surface = [1,2,3] # [delta_e, delta_a, delta_r]
control_prop = 100 # delta_t

ned = [0,0,0]
uvw = [1,2,3]
euler = [0,0,0]
pqr = [0.1,0.2,0.1]
states = [ned, uvw, euler, pqr]

lon_coef, lat_coef = get_aerodynamic_coefficients(lon_linear_coef, lat_linear_coef, MEAN_CHORD, WINGSPAN, alpha, beta, airspeed, pqr, control_surface)

aero = dotdict()
aero.fx, aero.fz, aero.m = longitudinal_aerodynamics(AIR_DENSITY, PLATFORM_AREA, MEAN_CHORD, alpha, airspeed, lon_coef)
aero.fy, aero.l, aero.n = lateral_aerodynamics(AIR_DENSITY, PLATFORM_AREA, WINGSPAN, airspeed, lat_coef)
grav = dotdict()
grav.fx, grav.fy, grav.fz = get_gravity_force(GRAV_ACC, MASS, euler)
prop = dotdict()
prop.fx, prop.fy, prop.fz = get_propeller_force(AIR_DENSITY, SWEPT, PROP_CHORD, MOTOR_FORCE_COEF, airspeed, control_prop)
prop.l, prop.m, prop.n = get_propeller_moment(MOTOR_TORQUE_COEF, PROP_SPEED_COEF, control_prop)

forces = np.sum([[v.fx, v.fy, v.fz] for v in [aero, grav, prop]], 0) # [fx, fy, fz]
moments = np.sum([[v.l, v.m, v.n] for v in [aero, prop]], 0) # [l, m, n]

states_dot = get_dynamics(states, MASS, MOMENT_OF_INERTIA, Gamma_i, forces, moments)
for i in states_dot:
    print(i)

[[1.]
 [2.]
 [3.]]
[[ -3659.39735758]
 [ 30150.2       ]
 [-14570.25394875]]
[[0.1]
 [0.2]
 [0.1]]
[[10907070.70888889]
 [  672736.29385641]
 [19090707.06888889]]


Coefficient conversion

In [197]:
# convert coefficients w.r.t. lmn(moments) into pqr(angular rate)
def convert_lmn2pqr(Gamma_i, ROLL_MOMENT_COEFFICIENTS, YAW_MOMENT_COEFFICIENTS):
    Cl0, Clb, Clp, Clr, Clda, Cldr = ROLL_MOMENT_COEFFICIENTS
    Cn0, Cnb, Cnp, Cnr, Cnda, Cndr = YAW_MOMENT_COEFFICIENTS
    G1, G2, G3, G4, G5, G6, G7, G8 = Gamma_i
    Cp0 = G3*Cl0 + G4*Cn0
    Cpb = G3*Clb + G4*Cnb
    Cpp = G3*Clp + G4*Cnp
    Cpr = G3*Clr + G4*Cnr
    Cpda = G3*Clda + G4*Cnda
    Cpdr = G3*Cldr + G4*Cldr
    Cr0 = G4*Cl0 + G8*Cn0
    Crb = G4*Clb + G8*Cnb
    Crp = G4*Clp + G8*Cnp
    Crr = G4*Clr + G8*Cnr
    Crda = G4*Clda + G8*Cnda
    Crdr = G4*Cldr + G8*Cndr
    Cp = [Cp0, Cpb, Cpp, Cpr, Cpda, Cpdr]
    Cr = [Cr0, Crb, Crp, Crr, Crda, Crdr]
    return Cp, Cr

Cp, Cr = convert_lmn2pqr(Gamma_i, ROLL_MOMENT_COEFFICIENTS, YAW_MOMENT_COEFFICIENTS)
print(Cp, Cr)

[0.1111111111111111, -0.09090909090909091, -0.09090909090909091, 0.09090909090909091, 0.1111111111111111, 0.1111111111111111] [0.1111111111111111, 0.09090909090909091, 0.09090909090909091, -0.09090909090909091, 0.1111111111111111, 0.1111111111111111]


5.4. Transfer Function Models

**Roll Dynamics Transfer Function**

$$
\dot{\phi} = p + d_{\phi_1}
$$

$$
\phi(s) = \left( \frac{a_{\phi_2}}{s(s+a_{\phi_1})}\right) \left( \delta_a(s) + \frac{1}{a_{\phi_2}} d_{\phi_2} (s)\right)
$$

In [205]:
# Roll Angle
def get_roll_coef(params, Cp, airspeed, sideslip, euler, pqr, delta_r):
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    Cp0, Cpb, Cpp, Cpr, Cpda, Cpdr = Cp
    G1, G2, G3, G4, G5, G6, G7, G8 = params.Gamma_i
    beta = sideslip
    phi, theta, psi = np.reshape(euler, (3))
    p, q, r = np.reshape(pqr, (3))
    a_phi_1 = -0.5*rho*airspeed**2*S*b*Cpp*b*0.5/airspeed
    a_phi_2 = 0.5*rho*airspeed**2*S*b*Cpda
    d_phi_1 = q*sin(phi)*tan(theta)+r*cos(phi)*tan(theta)
    d_phi_1_dot = 0 # Approximation
    d_phi_2 = G1*p*q - G2*q*r + 0.5*rho*airspeed**2*S*b*(Cp0 + Cpb*beta - 0.5*Cpp*b*d_phi_1/airspeed + 0.5*Cpr*b*r/airspeed + Cpdr*delta_r) + d_phi_1_dot
    return a_phi_1, a_phi_2, d_phi_2

roll_coef = get_roll_coef(params, Cp, airspeed, beta, euler, pqr, control_surface[-1])
print(roll_coef)


(204545.45454545456, 1666666.6666666665, 6687121.213939394)


**Course and Heading**

$$
\dot{\chi} = \frac{g}{V_G} \tan\phi
$$

$$
\chi(s) = \frac{g/V_g}{s} (\phi(s) + d_\chi (s))
$$

Alternatively, using GPS, 

$$
\psi(s) = \frac{g/V_a}{s} (\phi(s) + d_\chi (s))
$$

**Sideslip**

$$
\dot{\beta} = -a_{\beta_1} \beta + a_{\beta_2} \delta_r + d_\beta
$$

$$
\beta(s) = \frac{a_{\beta_2}}{s+a_{\beta_1}} (\delta_r(s) + d_\beta (s))
$$

In [207]:
def get_sideslip_coef(params, CY, airspeed, uvw, euler, pqr, delta_a):
    # Page 71.
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    MASS = params.MASS
    CY0, CYb, CYp, CYr, CYda, CYdr = np.reshape(CY, (6))
    u, v, w = np.reshape(uvw, (3))
    phi, theta, psi = np.reshape(euler, (3))
    p, q, r = np.reshape(pqr, (3))
    a_beta_1 = -0.5*rho*airspeed*S/MASS*CYb
    a_beta_2 = 0.5*rho*airspeed*S/MASS*CYdr
    d_beta = 1/airspeed*(p*w - r*u + params.GRAV_ACC*cos(theta)*sin(phi)) + 0.5*rho*airspeed*S/MASS*(CY0 + 0.5*CYp*b*p/airspeed + 0.5*CYr*b*r/airspeed + CYda*delta_a)
    return a_beta_1, a_beta_2, d_beta

CY = LATERAL_FORCE_COEFFICIENTS
sideslip_coef = get_sideslip_coef(params, CY, airspeed, uvw, euler, pqr, control_surface[1])
print(sideslip_coef)

(-50.0, 50.0, 151.502)


**Pitch Angle**

$$
\ddot{\theta} = -a_{\theta_1} \dot{\theta} - a_{\theta_2}\theta + a_{\theta_3} \delta_e + d_{\theta_2}
$$

In [210]:
def get_pitch_coef(params, airspeed, AOA, Cm):
    # Page 73.
    rho = params.AIR_DENSITY
    S = params.PLATFORM_AREA
    b = params.WINGSPAN
    c = params.MEAN_CHORD
    MASS = params.MASS
    Jy = params.MOMENT_OF_INERTIA[1]
    G1, G2, G3, G4, G5, G6, G7, G8 = params.Gamma_i
    Cm0, Cma, Cmq, Cmde = np.reshape(Cm, (4)) # [C_m_0, C_m_alpha, C_m_q, C_m_delta_e]
    u, v, w = np.reshape(uvw, (3))
    phi, theta, psi = np.reshape(euler, (3))
    p, q, r = np.reshape(pqr, (3))
    a_theta_1 = -0.25*rho*airspeed*c*S*Cmq*c/Jy
    a_theta_2 = -0.5*rho*airspeed**2*c*S*Cma/Jy
    a_theta_3 = 0.5*rho*airspeed**2*c*S*Cmde/Jy
    d_theta_1 = q*(cos(phi)-1)-r*sin(phi)
    d_theta_1_dot = 0 # Approximation
    gamma = theta - AOA
    d_theta_2 = G6*(r**2-p**2) + G5*p*r + 0.5*rho*airspeed**2*c*S/Jy*(Cm0 - Cma*gamma - 0.5*Cmq*c/airspeed*d_theta_1) + d_theta_1_dot
    return a_theta_1, a_theta_2, a_theta_3, d_theta_2

Cm = PITCHING_MOMENT_COEFFICIENTS
pitch_coef = get_pitch_coef(params, airspeed, alpha, Cm)
print(pitch_coef)

(4000.0, 200000.0, 200000.0, 137168.14692820414)


**Altitude**

$$
\dot{h} = V_a \theta + d_h\\
d_h = (u\sin\theta - V_a \theta) - v \sin \phi \cos\theta - w \cos\phi \cos\theta
$$

$$
h(s) = \frac{V_a}{S} \left( \theta + \frac{1}{V_a} dh \right)
$$

**Airspeed**

$$
\dot{V}_a = \dot{u}\cos\alpha + \dot{w} \sin \alpha + d_{V_1}\\
d_{V_1} = -\dot{u}(1-\cos\beta)\cos\alpha - \dot{w}(1-\cos\beta)\sin\alpha + \dot{v}\sin\beta
$$

$$
\bar{V}_a(s) = \frac{1}{s+a_{V_1}} ( a_{V_2} \bar{\delta}_t (s) - a_{V_3}\bar{\theta}(s) + d_{V}(s))
$$

In [ ]:
def get_airspeed_coef(params, airspeed, CD):
    